In [48]:
import os 
import numpy as np 
import pandas as pd
from collections import defaultdict
import itertools
import ast

In [66]:
#hyper parameters
rating_threshold = 4.5 #only counted the usr rating if it is rated above the rating threshold
user_count_threshold = 500 #only keep the bk_id if the bk is rated by user for more than user_count_threshold times
split = [0.8, 0.1, 0.1] #train, dev, test split
#INPUT_DIR = '../../../data/the-movies-dataset/'
INPUT_DIR ='../../data/the-movies-dataset/'
OUTPUT_DIR = '../box-code/data/movie_data/movie_big/taxonomy'

In [50]:
#read in raw data
#prepare data for the complete dataset
ratings_file = INPUT_DIR + 'ratings.csv'
df = pd.read_csv(ratings_file, delimiter=',')
df.dataframeName = 'ratings.csv'

In [51]:
#preprocess data
df.drop(columns=['timestamp'])
df = df[df.rating >= rating_threshold]#Drop all entry with rating lower than the threshold

movie_dict = defaultdict(list)#dict with key: movie_id, val: the list of user_ids rated the movie above the rating threshold
for index, row in df.iterrows():    
    movieId = int(row['movieId'])
    usrId = int(row['userId'])
    movie_dict[movieId].append(usrId)

for key in list(movie_dict.keys()):#Drop all movie entry if number of rates below the user count threshold 
    if len(movie_dict[key]) < user_count_threshold:
        del movie_dict[key]

In [71]:
movie_id_set = set(movie_dict.keys())#make a set of valid movie ids used in our data after filtering
print(len(movie_id_set))


1817


In [72]:
#extract genre ID as integers from data
def extractGenreID(text, MAX_MV_ID ):
    tmp = ast.literal_eval(text)
    #ids = []
    gen = []
    for d in tmp:
       # ids.append(int(d['id'])+MAX_MV_ID)
        gen.append(d['name'])
   # return [ids, gen]
    return gen

In [73]:
#dict of movie and labeled genre pairs, key: movie id, value: list of genre id
MAX_MV_ID = 163949+1
mv_genre = {}
genres = {} #dict for genres, key: genre id, value: genre name in English
for i in range(len(df_metadata)):
    try:
        this_mv_id = int(df_metadata.loc[i].id)
        this_mv_gen = extractGenreID(df_metadata.loc[i].genres, MAX_MV_ID)
        '''
        this_mv_genre_list = ids  
        for i in range(len(ids)):
            genres[ids[i]] = gen[i] #have tested every genre and genre_id pair are unique
        '''
        mv_genre[this_mv_id ] = this_mv_gen #have tested every mv_id only appear once in the dataframe        
    except ValueError:
        #print(i, df_metadata.loc[i].id,df_metadata.loc[i].genres )
        print("error in data")

error in data
error in data
error in data


In [74]:
for k, v in mv_genre.items():
    print(k,v)

862 ['Animation', 'Comedy', 'Family']
8844 ['Adventure', 'Fantasy', 'Family']
15602 ['Romance', 'Comedy']
31357 ['Comedy', 'Drama', 'Romance']
11862 ['Comedy']
949 ['Action', 'Crime', 'Drama', 'Thriller']
11860 ['Comedy', 'Romance']
45325 ['Action', 'Adventure', 'Drama', 'Family']
9091 ['Action', 'Adventure', 'Thriller']
710 ['Adventure', 'Action', 'Thriller']
9087 ['Comedy', 'Drama', 'Romance']
12110 ['Comedy', 'Horror']
21032 ['Family', 'Animation', 'Adventure']
10858 ['History', 'Drama']
1408 ['Action', 'Adventure']
524 ['Drama', 'Crime']
4584 ['Drama', 'Romance']
5 ['Crime', 'Comedy']
9273 ['Crime', 'Comedy', 'Adventure']
11517 ['Action', 'Comedy', 'Crime']
8012 ['Comedy', 'Thriller', 'Crime']
1710 ['Drama', 'Thriller']
9691 ['Action', 'Adventure', 'Crime', 'Thriller']
12665 ['Drama', 'Fantasy', 'Science Fiction', 'Thriller']
451 ['Drama', 'Romance']
16420 ['Drama']
9263 ['Comedy', 'Drama', 'Family']
17015 ['Drama', 'Romance']
902 ['Fantasy', 'Science Fiction', 'Adventure']
37557 [

32574 ['Drama', 'Comedy', 'Romance']
22657 ['Drama', 'Action', 'Mystery', 'Romance']
76464 ['Comedy']
22649 ['Drama', 'Romance', 'War']
176841 ['Music', 'Mystery']
125587 ['Drama']
39448 ['Action', 'Foreign']
52855 ['Drama']
39940 ['Drama', 'Romance']
172868 []
85328 ['Comedy', 'Romance', 'Drama']
124632 ['Documentary', 'Family']
11076 ['Action', 'Adventure', 'Drama', 'Family']
63945 ['Drama', 'Romance']
58985 ['Drama', 'Comedy', 'Music']
890 ['Drama']
10861 ['Action', 'Adventure', 'Thriller']
1770 ['Drama', 'Thriller']
44465 ['Drama', 'Thriller']
2033 ['Drama', 'Romance']
18203 ['Drama']
9333 ['Action', 'Crime', 'Drama', 'Thriller']
47260 ['Action', 'Drama', 'Thriller']
9400 ['Drama', 'Action', 'Crime']
9401 ['Comedy', 'Crime']
12241 ['Comedy', 'Crime', 'Thriller']
25739 ['Comedy']
161806 []
11306 ['Drama', 'Thriller']
9625 ['Action', 'Adventure', 'Crime', 'Drama', 'Thriller']
10680 ['Action', 'Comedy', 'Drama', 'Family']
6346 ['Crime', 'Drama']
18660 ['Comedy', 'Family']
35115 ['Acti

3116 ['Drama']
1051 ['Action', 'Crime', 'Thriller']
1366 ['Drama']
12102 ['Drama']
16619 ['Drama']
9443 ['Drama', 'History']
11050 ['Drama', 'Comedy']
606 ['Drama']
746 ['Drama', 'History']
380 ['Drama']
403 ['Comedy', 'Drama']
11485 ['Comedy', 'Crime']
466 ['Mystery']
13820 ['Comedy', 'Crime', 'Science Fiction', 'Thriller']
15389 ['Comedy', 'Drama']
13597 ['Adventure', 'Family', 'Fantasy']
2108 ['Comedy', 'Drama']
10014 ['Horror']
10072 ['Horror', 'Thriller']
10131 ['Horror', 'Thriller']
10160 ['Horror', 'Thriller']
11284 ['Horror', 'Thriller', 'Comedy']
4488 ['Horror']
9725 ['Horror', 'Thriller']
9728 ['Horror', 'Thriller']
9730 ['Horror', 'Thriller']
9731 ['Horror', 'Thriller']
10225 ['Horror', 'Mystery', 'Thriller']
10281 ['Horror', 'Thriller']
10283 ['Horror', 'Thriller']
948 ['Horror', 'Thriller']
11281 ['Horror', 'Thriller']
10676 ['Horror', 'Mystery', 'Science Fiction']
11357 ['Horror', 'Thriller']
11361 ['Horror', 'Thriller']
36599 ['Horror', 'Mystery', 'Thriller']
39929 ['Hor

253 ['Adventure', 'Action', 'Thriller']
13511 ['Horror']
660 ['Adventure', 'Action', 'Thriller']
46798 ['Drama']
11377 ['Horror', 'Mystery', 'Thriller']
26149 ['Drama']
492 ['Fantasy', 'Drama', 'Comedy']
108346 ['Romance', 'Drama']
98480 ['Romance', 'Comedy']
128 ['Adventure', 'Fantasy', 'Animation']
46286 ['Comedy', 'Drama']
8672 ['Documentary']
8056 ['Drama', 'Comedy', 'War']
2020 ['Comedy']
9481 ['Drama', 'Mystery', 'Thriller', 'Crime']
9008 ['Drama', 'Thriller']
14242 ['Documentary']
16129 ['Romance', 'Drama', 'Science Fiction']
104103 ['Drama', 'Comedy']
11489 ['Drama']
28145 ['Drama']
51371 ['Drama', 'Comedy']
18111 ['Comedy', 'Horror', 'Science Fiction']
31361 ['Adventure', 'Comedy']
11223 ['Thriller', 'Mystery', 'Drama']
16281 ['Horror', 'Comedy', 'Fantasy']
16288 ['Horror']
1694 ['Science Fiction', 'Comedy', 'Horror']
476 ['Crime', 'Drama']
37094 ['Crime', 'Drama', 'Thriller']
13555 ['Horror']
961 ['Action', 'Adventure', 'Comedy', 'Drama']
73969 ['Drama', 'Comedy', 'Romance']


13898 ['Drama', 'Foreign']
196280 []
9290 ['Adventure', 'Comedy']
13166 ['Comedy']
32590 ['Drama', 'Romance']
109472 ['Drama', 'Thriller', 'Romance']
153141 ['Romance', 'Comedy', 'Drama']
52654 ['Romance', 'Drama']
70282 ['Drama']
47643 ['Drama', 'Fantasy', 'Romance']
111 ['Action', 'Crime', 'Drama', 'Thriller']
32488 ['Drama', 'Romance']
10477 ['Action']
12484 ['Action', 'Adventure', 'Horror', 'Thriller']
2144 ['Action', 'Comedy', 'Crime']
24113 ['Comedy', 'Romance']
118052 ['Drama', 'Fantasy', 'Comedy', 'Family']
1734 ['Adventure', 'Action', 'Fantasy']
38047 ['Drama']
8076 ['Fantasy', 'Drama', 'Comedy', 'Science Fiction', 'Romance', 'Family', 'Foreign']
4973 ['Drama', 'Mystery']
8095 ['Drama', 'History', 'Romance']
849 ['Fantasy', 'Action', 'Adventure']
27223 ['Comedy']
2981 ['Adventure', 'Drama', 'Fantasy', 'Science Fiction']
39266 ['Documentary']
31863 ['Crime', 'Drama']
11307 ['Comedy']
46498 ['Fantasy', 'Comedy', 'Adventure']
11163 ['Drama', 'Fantasy']
11035 ['Comedy', 'Drama']
1

...

389427 ['Comedy']
144341 []
147575 ['Western']
150224 ['Western']
43074 ['Action', 'Fantasy', 'Comedy']
52947 ['Action', 'Comedy', 'Foreign']
339739 ['Drama']
61294 ['Drama', 'Thriller', 'Romance']
30942 ['Action']
32003 ['Horror']
353610 ['Action']
350566 []
206657 ['Documentary', 'Music']
287703 ['Comedy', 'Romance']
284152 ['Crime']
25625 ['Crime', 'Comedy', 'Horror']
168098 ['Horror', 'Science Fiction', 'Thriller']
356334 ['Action']
28529 ['Fantasy', 'Horror', 'Thriller']
300210 ['Documentary']
112767 ['Thriller', 'Foreign']
131172 ['Comedy']
377150 []
244839 ['Action', 'Science Fiction']
216521 []
170898 ['Comedy', 'Romance']
25079 ['Animation', 'Family']
80699 ['Drama']
130638 ['Documentary']
363889 ['Comedy', 'Horror']
355598 ['Drama', 'Horror', 'Thriller']
323372 ['Drama', 'Thriller']
370687 ['Fantasy', 'Action', 'Adventure']
373476 ['Drama', 'Mystery', 'Thriller']
300761 ['Documentary', 'TV Movie']
41312 ['Drama', 'War']
258755 ['Romance', 'Drama']
28775 ['Horror', 'Science Fi

311215 ['Comedy', 'Drama']
250278 ['Mystery', 'Thriller']
68972 ['Comedy']
286313 ['Drama']
125276 ['Drama']
244331 ['Documentary']
73306 ['Horror', 'Drama']
34745 ['Action', 'Comedy', 'Drama']
418072 ['Crime', 'Action', 'Adventure']
182548 ['Documentary']
60504 ['Thriller', 'Crime', 'Horror']
268853 ['Drama']
73823 ['Comedy']
52204 ['Science Fiction', 'Horror']
42793 ['Horror', 'Science Fiction']
62722 ['Crime', 'Drama', 'Mystery', 'Romance']
44805 ['Comedy']
403434 ['Thriller']
375366 ['Drama', 'Horror', 'Thriller']
408537 ['Comedy', 'Drama']
57278 ['Adventure', 'Fantasy', 'Foreign', 'Science Fiction']
245627 ['Horror', 'Thriller']
298583 ['Comedy', 'Romance']
388764 ['Comedy', 'Action', 'Adventure']
124501 ['Horror', 'Mystery', 'Drama']
389396 ['Drama', 'Crime', 'Mystery']
399300 ['Action']
276624 ['Action', 'Science Fiction', 'Fantasy', 'Horror']
377447 ['Action', 'Comedy', 'War']
408620 ['Action', 'Crime']
410729 ['Thriller']
379300 ['Thriller', 'Fantasy']
413198 ['Adventure', 'Ac

35992 ['Comedy']
332665 []
428645 ['Comedy']
409082 ['Comedy']
380622 ['Drama']
358901 ['Crime']
362202 ['Crime', 'Thriller']
362180 ['Drama', 'Horror', 'Science Fiction']
362585 ['Animation', 'Fantasy', 'Mystery']
150140 ['TV Movie', 'Drama', 'Family']
227932 ['Comedy']
377847 ['Comedy']
335897 ['Comedy']
286545 ['Crime', 'Comedy']
128044 []
19054 ['Documentary']
114145 []
411678 ['Drama']
424782 ['Drama', 'History', 'Crime', 'Mystery', 'Thriller']
414937 ['Mystery', 'Drama', 'Crime']
401222 ['Thriller', 'Adventure', 'Science Fiction', 'Mystery']
324552 ['Thriller', 'Action', 'Crime']
419430 ['Mystery', 'Thriller', 'Horror']
263115 ['Action', 'Drama', 'Science Fiction']
293167 ['Action', 'Adventure', 'Fantasy']
43989 ['Comedy']
201430 ['Comedy', 'Fantasy', 'Horror', 'Science Fiction']
408626 ['Crime', 'Drama', 'Thriller']
180863 ['Crime', 'Drama']
105352 ['Fantasy', 'Drama']
179302 ['Animation', 'Foreign']
105388 ['Drama', 'Documentary', 'Family']
295011 ['Action', 'Drama', 'Horror', 

45318 ['Documentary']
77710 ['Adventure', 'Comedy']
161244 ['Drama', 'Romance']
254268 ['Animation', 'Fantasy']
144726 ['Comedy']
52779 ['Documentary']
363690 ['Thriller', 'Horror', 'Action']
122487 ['Drama']
11960 ['Drama', 'Foreign', 'Romance']
62842 ['Drama', 'Documentary', 'Foreign']
172295 ['Drama']
106259 ['Drama', 'War']
402976 ['Horror', 'Comedy']
445916 ['Horror']
387399 ['Drama', 'Horror', 'Thriller']
289269 ['Action', 'Thriller']
322148 ['Comedy', 'Horror']
389578 ['Science Fiction', 'Action']
256608 ['Comedy', 'Science Fiction', 'War']
187010 ['Comedy', 'Horror']
207260 ['Horror', 'Mystery', 'Thriller']
164377 ['Action', 'Horror', 'Science Fiction']
110608 ['Drama', 'War', 'Mystery']
67308 ['Drama']
55429 ['Horror', 'Comedy']
109354 ['Comedy']
354133 ['Horror']
371833 ['Drama', 'Horror']
76638 ['Animation', 'Comedy', 'Fantasy']
41581 ['Adventure', 'Animation', 'Comedy']
309898 ['Adventure', 'Family', 'Comedy']
331641 ['Drama']
83495 ['Drama', 'Fantasy', 'Science Fiction']
7

27018 ['Drama']
61008 ['Comedy', 'Foreign']
424643 ['Horror']
255391 ['Comedy', 'Drama']
116336 ['Adventure', 'Science Fiction']
446345 ['Romance', 'Comedy']
429200 ['Drama', 'Thriller', 'Crime']
37991 ['Comedy']
384669 []
412758 ['Action', 'Comedy', 'Horror']
331110 ['Drama', 'Crime', 'Thriller']
436882 ['Romance', 'Drama']
184363 ['Documentary']
445840 ['Comedy']
92475 ['Documentary']
189505 ['Documentary']
176407 ['Animation']
2286 ['Adventure', 'Fantasy', 'History']
438081 ['Comedy', 'Drama']
449742 ['Thriller', 'Drama']
187602 []
427673 ['Comedy']
149723 ['Drama']
453008 ['Documentary']
345489 ['Drama']
84626 ['Family', 'Science Fiction']
86337 ['Action', 'Adventure', 'Science Fiction', 'Fantasy']
345914 ['Comedy']
257698 ['Comedy', 'Drama']
449758 ['War', 'Documentary']
440143 ['Romance', 'Action', 'Adventure', 'Comedy', 'Drama']
433054 ['Documentary']
381519 ['Action', 'Animation', 'Science Fiction']
438634 ['Drama', 'Family']
50387 ['Drama', 'Comedy', 'Adventure']
143068 []
495

In [75]:
print(len(mv_genre))
for k in list(mv_genre.keys()):#double check if all movie_genre_pairs are the valid movies after filtering
    if k not in movie_id_set:
        del mv_genre[k]
print(len(mv_genre))

45433
736


In [76]:
#create the movie_genre_master file
file = open(OUTPUT_DIR+"movie_genre_master.txt", "w") 
for movie_id in list(mv_genre.keys()):
    for genre_id in mv_genre[movie_id]:
        file.write("IsA\t"+str(movie_id)+"\t" + str(genre_id) + "\t" + str(1)+"\n")
file.close() 

In [63]:
user_dict = defaultdict(list)#dict with key: user, val: the movie_id this usr rated above given threshold
for key, val in movie_dict.items():
    for i in val:
        user_dict[i].append(key)
        
#to make sure if a user rate a movie, the user only rate the movie once
for key, val in user_dict.items():
    user_dict[key] = list(set(val))

#movie_movie_count: key: (bk_id_i, bk_id_j) (bk_id_i != bk_id_j and bk_id_i < bk_id_j) 
#                 val: co-ocurrence count of bk_id_i and bk_id_j
#movie_movie_count_marginal: key: (bk_id_i) val: ocurrence count of bk_id_i in the dataset
movie_movie_count =  defaultdict(lambda: 0)
movie_movie_count_marginal = defaultdict(lambda:0)
for key, val in user_dict.items():
    for i in range(len(val)):
        movie_movie_count_marginal[val[i]] += 1 #marginal count
        for j in range(i+1, len(val)):#marginal count not included in book_book_count
            movie_movie_count[(val[i], val[j]) if val[i] <= val[j] else (val[j], val[i])] += 1
            

In [ ]:
genre_movie_dict = defaultdict(list)#dict with key: user, val: the movie_id this usr rated above given threshold
for key, val in movie_dict.items():
    for i in val:
        user_dict[i].append(key)


In [47]:
for k, v in genres.items():
    print(k,v)

163965 Animation
163984 Comedy
174700 Family
163961 Adventure
163963 Fantasy
174698 Romance
163967 Drama
163977 Action
164029 Crime
164002 Thriller
163976 Horror
163985 History
164827 Science Fiction
173597 Mystery
174701 War
174718 Foreign
174351 Music
164048 Documentary
163986 Western
174719 TV Movie
